# Platform Data Analysis

## Execution

### Import packages

In [59]:
# pip3 install mariadb
# pip3 install pandas
# pip3 install numpy  
# sudo apt-get install libatlas-base-dev
# pip3 install matplotlib
# pip3 install ipympl
# pip3 install ipywidgets

%matplotlib widget
%config InlineBackend.figure_format = 'png'

import os
import sys
import re
import mariadb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual, IntSlider, IntRangeSlider, HBox
import ipywidgets as widgets

### Define Objects

In [60]:
class _ss_dict:
    def __init__(self, span: tuple, avg_value: float):
        self.span=span
        self.avg_value=avg_value
    def gen_series(self, leng: int,): # series of avg_value @ certain time span
        _series = pd.Series(0, index=range(leng))
        for i in range(self.span[0], self.span[-1]):
            _series[i] = self.avg_value
        self.series = _series
    def __repr__(self):
        rep = f'Steady State@{self.span}, avg: {self.avg_value}'
        return rep

class SerEnergy_Set_Point:
    density_H2 = 0.08988 # kg/m3 @ STP
    enthalpy_H2 = 120.21 # MJ/kg-LHV
    enthalpy_MeOH = 20.09 # MJ/kg-LHV
    MeOH_weight = 0.543 # 54.3wt%
    cont_step = 60 #continue for 1 min
    TC_range = 5 #5oC
    Scale_range = 5 #5g/min
    continuity = 10 #10min
    def __init__(self, name: str, temp: float, weight_rate: float, dummy: float):
        self.name = name
        # set points
        self.temp = temp 
        self.weight_rate = weight_rate
        self.dummy = dummy
        
    def cond(self, df: pd.core.frame.DataFrame, TC: str, Scale: str):
        self.ss_time = []
        count_60 = 0 #counter for steady mins
        ss_temp = []
        ss_lst = []
        ss = ((df[TC] >= (self.temp - self.TC_range)) & (df[TC] <= (self.temp + self.TC_range))) \
            & ((df[Scale] >= (self.weight_rate - self.Scale_range)) & (df[Scale] <= (self.weight_rate + self.Scale_range)))
        #print(ss.sum())
        for i in range(0, len(ss), self.cont_step):
            if i + self.cont_step <= len(ss):
                if ss.iloc[i:i + self.cont_step].sum() == 60:
                    count_60 += 1
                    ss_temp.append(i)
                else:
                    count_60 = 0
                    if ss_temp != []:
                        ss_lst.append(ss_temp)
                        ss_temp = []
                    continue
        #print(ss_lst)
        for u in ss_lst:
            if len(u) >= self.continuity:
                self.ss_time.append((u[0], u[-1]))
        self.ss_time = tuple(self.ss_time)
    def eff_calc(self, df: pd.core.frame.DataFrame, TC10: str, TC11: str, EVA_Out: str, Scale: str, DFM_RichGas: str, GA_H2: str):
        if self.ss_time:
            self.ss_TC10 = []
            self.ss_TC11 = []
            self.ss_EVA_Out = []
            self.ss_weight_rate = []
            self.ss_DFM_RichGas = []
            self.ss_GA_H2 = []
            self.heff = []
            for u in self.ss_time:
                #print(u)
                ls = [v for v in range(u[0],u[-1])]
                df_mean = df[df.index.isin(ls)]
                _avg_TC10 = df_mean[TC10].mean()
                _avg_TC11 = df_mean[TC11].mean()
                _avg_EVA_Out = df_mean[EVA_Out].mean()
                _avg_Scale = df_mean[Scale].mean()
                _avg_DFM_RichGas = df_mean[DFM_RichGas].mean()
                _avg_GA_H2 = df_mean[GA_H2].mean()
                self.ss_TC10.append(_ss_dict(span=u, avg_value=_avg_TC10))
                self.ss_TC11.append(_ss_dict(span=u, avg_value=_avg_TC11))
                self.ss_EVA_Out.append(_ss_dict(span=u, avg_value=_avg_EVA_Out))
                self.ss_weight_rate.append(_ss_dict(span=u, avg_value=_avg_Scale))
                self.ss_DFM_RichGas.append(_ss_dict(span=u, avg_value=_avg_DFM_RichGas))
                self.ss_GA_H2.append(_ss_dict(span=u, avg_value=_avg_GA_H2))
                _avg_H2_flow = self.density_H2 * ((_avg_DFM_RichGas * _avg_GA_H2) / 100 - self.dummy) / 1000 # kg/min @ STP
                _heff = ((_avg_H2_flow * self.enthalpy_H2) / (_avg_Scale / 1000 * self.MeOH_weight * self.enthalpy_MeOH)) * 100 # enthalpy eff [%]
                self.heff.append(_ss_dict(span=u, avg_value=_heff))
            self.ss_TC10 = tuple(self.ss_TC10)
            self.ss_TC11 = tuple(self.ss_TC11)
            self.ss_EVA_Out = tuple(self.ss_EVA_Out)
            self.ss_weight_rate = tuple(self.ss_weight_rate)
            self.ss_DFM_RichGas = tuple(self.ss_DFM_RichGas)
            self.ss_GA_H2 = tuple(self.ss_GA_H2)
            self.heff = tuple(self.heff)
            #print(self.name, self.ss_weight_rate, self.ss_DFM_RichGas, self.ss_GA_H2)
            #print(self.name, self.heff)
        else:
            pass
    def gen_dataframe(self): # dataframe of summaries
        if self.ss_time:
            _sum_rows = []
            for i in range(len(self.ss_time)):
                _sum_rows.append(pd.DataFrame([[self.name, self.ss_time[i][0], self.ss_time[i][-1], round(self.ss_TC10[i].avg_value, 2), round(self.ss_TC11[i].avg_value, 2), round(self.ss_EVA_Out[i].avg_value, 2), round(self.ss_weight_rate[i].avg_value, 2), round(self.ss_DFM_RichGas[i].avg_value, 2), round(self.ss_GA_H2[i].avg_value, 2), round(self.heff[i].avg_value, 2)]], 
                                                 columns=['Steady State', 'Init[s]', 'End[s]', 'TC10[oC]', 'TC11[oC]', 'EVA_Out[oC]', 'Avg_WeightRate[g/min]', 'Avg_RichGas[LPM]', 'Avg_H2_Conc[%]', 'Enthalpy_Eff[%]'])
                                   )
            self.sum_rows = pd.concat(_sum_rows)
        else:
            pass
        

### Define Functions

In [158]:
def plot(Table_name: str, Time: tuple, SS: bool, Calc: bool, Ploting: bool):
    if Calc or Ploting:
        #print(Table_name, Time)
        cur.execute(f"SELECT * FROM {Table_name}")
        df = pd.DataFrame(cur.fetchall(), columns=[entry[0] for entry in cur.description]).set_index('Id')
        _leng = len(df)
            
        _bls=[]
        _heffls = 0
        for i in Set_Point_lst:
            i.cond(df, TC='TC10', Scale='Scale')
            i.eff_calc(df, TC10='TC10', TC11='TC11', EVA_Out='EVA_Out', Scale='Scale', DFM_RichGas='DFM_RichGas', GA_H2='GA_H2')
            if i.ss_time:
                for u in i.ss_time:
                    #print(u)
                    for v in range(u[0], u[-1]):
                        _bls.append(v)
                for q in i.heff:
                    q.gen_series(leng=_leng)
                    #print(q.series.sum())
                    _heffls = _heffls + q.series
        #print(_effls.sum())
        
        df_sum = pd.DataFrame()
        for i in Set_Point_lst:
            i.gen_dataframe()
            if i.ss_time:
                #print(i.sum_rows)
                df_sum = pd.concat([df_sum, i.sum_rows])
        if not df_sum.empty:
            df_sum = df_sum.sort_values(by=['Init[s]']).reset_index(drop=True)
            display(df_sum)
        else:
            print('No Steady-State is found!')
        
        if SS:        
            _bmask = pd.Series([True if i in _bls else False for i in range(0, _leng)])
            df = df.where(_bmask, 0)
        else:
            pass
    
    if Ploting:
        #plt.clf()
        fig, (ax_TC, ax_DFM, ax_GA, ax_heff) = plt.subplots(4, constrained_layout=True, figsize=(10, 10), sharex=False)
        fig.canvas.toolbar_position = 'left'
        
        if db_name == 'Reformer_SE':
            df['TC10'].plot(legend=True, ax=ax_TC, kind='area', color='lightblue')
            df['TC6'].plot(legend=True, ax=ax_TC)
            df['TC7'].plot(legend=True, ax=ax_TC)
            df['TC8'].plot(legend=True, ax=ax_TC)
            df['TC9'].plot(legend=True, ax=ax_TC)
            df['TC11'].plot(legend=True, ax=ax_TC)
            df['EVA_Out'].plot(legend=True, ax=ax_TC)
            ylim = (0,600)
            yticks = range(0,600,100)
        elif db_name == 'Reformer_BW':
            df['TC7'].plot(legend=True, ax=ax_TC, kind='area', color='lightblue')
            df['TC8'].plot(legend=True, ax=ax_TC)
            df['TC9'].plot(legend=True, ax=ax_TC)
            df['TC10'].plot(legend=True, ax=ax_TC)
            df['EVA_out'].plot(legend=True, ax=ax_TC)
            ylim = (0,1000)
            yticks = range(0,1000,100)
        
        ax_TC.set(title=f'TC_{Table_name}',
                  xlim=Time, 
                  xticks=range(Time[0],Time[1],1200),
                  ylabel='Temp[oC]',
                  xlabel='Time[s]',
                  ylim=ylim, 
                  yticks=yticks,
                 )
        ax_TC_2 = ax_TC.twinx()
        ax_TC_2.set(ylim=ylim, yticks=yticks,)
        
        df_DFM = pd.DataFrame({
            'DFM_AOG':df['DFM_AOG'],
            'DFM_RichGas':df['DFM_RichGas'],
        }, index=df.index)
        
        df_DFM.plot(legend=True, ax=ax_DFM, kind='area', stacked=True, 
                title=f'DFM_{Table_name}',
                ylabel='Gas Production [LPM]',
                grid=True,
                xlim=Time, 
                xticks=range(Time[0],Time[1],1200),
                ylim=(0,150), 
                yticks=range(0,150,10)
               )
        ax_DFM_2 = df['Scale'].plot(legend=True, ax=ax_DFM, secondary_y=True, xlabel='Time[s]',)
        ax_DFM_2.set_ylabel('Scale [g/min]')
        ax_DFM_2.set_ylim(0,100)
        
        df_GA = pd.DataFrame({
            'GA_H2':df['GA_H2'],
            'GA_CO':df['GA_CO'],
            'GA_CO2':df['GA_CO2'],
            'GA_CH4':df['GA_CH4'],
            'GA_N2':df['GA_N2'],
        }, index=df.index)
        
        df_GA.plot(legend=True, ax=ax_GA, kind='area', stacked=True, 
                title=f'GA_{Table_name}', 
                ylabel='Gas Composition [%]', 
                #grid=True,  
                xlim=Time, 
                xticks=range(Time[0],Time[1],1200),
                ylim=(0,100), 
                yticks=range(0,100,10)
               )
        df['CO_line'] = df['GA_CO']
        ax_GA_2 = df['CO_line'].plot(legend=True, ax=ax_GA, secondary_y=True, xlabel='Time[s]',)
        ax_GA_2.set_ylabel('CO [%]')
        ax_GA_2.set_ylim(0,5)
        
        if not df_sum.empty:
            _heffls.plot(legend=False, ax=ax_heff, kind='area', stacked=False, 
                    title=f'heff_{Table_name}', 
                    ylabel='Enthalpy Eff. [%]',
                     xlabel='Time[s]',
                    #grid=True,  
                    xlim=Time, 
                    xticks=range(Time[0],Time[1],1200),
                    ylim=(0,100), 
                    yticks=range(0,100,10)
                   )
        
        plt.show()
    
    else:
        pass

### Select a Database

In [159]:
def db_conn(username=username, password=password, db_name=db_name):
    try:
        # Connect to MariaDB Platform
        conn = mariadb.connect(
            user=username,
            password=password,
            host="localhost",
            port=3306,
            database=db_name,
            autocommit=True
        )

        # Get Cursor for tx
        cur = conn.cursor(named_tuple=False)
        #print(conn)
        print(f"Connected to {db_name}@localhost ")
        return cur
        
    except mariadb.Error as e:
        print(f"Error connecting to MariaDB Platform: {e}")
        sys.exit(1)
    

### Connect to a Remote Database

### Data Visualization 

In [160]:
Set_Point_lst = [SerEnergy_Set_Point(name= '18A', temp=300, weight_rate=18.91, dummy=0), 
                 SerEnergy_Set_Point(name= '34A', temp=305, weight_rate=31.85, dummy=0), 
                 SerEnergy_Set_Point(name= '51A', temp=315, weight_rate=49.02, dummy=9.5), 
                 SerEnergy_Set_Point(name= '68A', temp=325, weight_rate=63.1, dummy=0), 
                 SerEnergy_Set_Point(name= '85A', temp=332, weight_rate=81.45, dummy=12.5)]


username = os.environ.get("db_user")
password = os.environ.get("db_pwd")
db_name_button = widgets.RadioButtons(
        options=['Reformer_BW', 'Reformer_SE'],
        description='Select Database:',
        disabled=False
        )
db_name = db_name_button.value
cur = db_conn(username=username, password=password, db_name=db_name)
cur.execute('SHOW tables')

table_dropdown = widgets.Dropdown(options=[u for i in cur.fetchall()[:] for u in i][::-1], description='Table')

cur.execute(f"SELECT * FROM {table_dropdown.value}")
cur.fetchall()

time_slider = widgets.IntRangeSlider(value=[0, cur.rowcount], max=cur.rowcount, step=1200, description='Time Span', continuous_update=False)

checkbox = widgets.Checkbox(value=False, description='Steady-State Only', disabled=False, indent=False)
toggle_calc = widgets.ToggleButton(value=False, description='Only Calc', disabled=False, button_style='success', tooltip='Description',icon='check')
toggle_plot = widgets.ToggleButton(value=False, description='Plot it!', disabled=False, button_style='info', tooltip='Description',icon='check')
#TC_selector = widgets.RadioButtons(options=[i[0] for i in cur.description if re.search("^TC|^EVA", i[0])], description='Area plot', disabled=False,)

def update_db_name(change):
    global db_name
    global cur
    db_name = change.new
    cur = db_conn(username=username, password=password, db_name=db_name)
    cur.execute('SHOW tables')
    table_dropdown.options = [u for i in cur.fetchall()[:] for u in i][::-1]
    cur.execute(f"SELECT * FROM {table_dropdown.value}")
    cur.fetchall()
    time_slider.value = [0, cur.rowcount]
    time_slider.max = cur.rowcount
db_name_button.observe(update_db_name, 'value')    

'''
def update_range(*args):
    cur.execute(f"SELECT * FROM {table_dropdown.value}")
    cur.fetchall()
    time_slider.max = cur.rowcount
    #TC_selector.options = [i[0] for i in cur.description if re.search("^TC|^EVA", i[0])]
table_dropdown.observe(update_range, 'value')
'''

plot = interactive_output(plot, {'Table_name':table_dropdown, 'Time':time_slider, 'SS':checkbox, 'Calc': toggle_calc, 'Ploting':toggle_plot})
toggle = HBox([toggle_calc, toggle_plot])
control = HBox([table_dropdown, time_slider, checkbox,])
#selector = HBox([TC_selector])
widgets.VBox([db_name_button, toggle, control, plot])


Connected to Reformer_BW@localhost 


### Close Remote Connection 

In [133]:
conn.close()

## Summary

#### 實驗結果 
- 圖中有兩個穩定點(Steady-State), 分別為 ***半載(51A)*** 和 ***全載(85A)*** 的操作條件
    - 半載穩定時間區間:
        - **7680-8580s (約15分)**
    - 全載穩定時間區間: 
        - **9720-11340s (約27分)**
- 兩個穩定點的平均觸媒床溫度(TC10)，均在參考溫度附近:
    - 半載均溫: **317.28oC**, 參考溫度: **315oC**
    - 半載均溫: **333.22oC**, 參考溫度: **332oC**
- 燃料蒸氣的進料溫度(EVA_Out) 約落在**200-200oC**
- 平均燃料液體進料流率(Weight_Rate):
    - 半載: **49.54 [g/min]**, 參考: **49.02 [g/min]**
    - 半載: **81.75 [g/min]**, 參考: **81.45 [g/min]**
- 此次實驗，燃燒室的燃料全程使用純氫進料(未採用回燒管路)，流率分別為:
    - 半載: **9.5 [LPM]**
    - 半載: **12.5 [LPM]**
- 重組次單元的熱效率(Enyhalpy Efficiency)分別為:
    - 半載: **94.45 %**
    - 半載: **93.59 %**

#### 與車測中心比較
- 重組次單元的熱效率(Enyhalpy Efficiency)分別為:
    - 半載: **88.1 %** (此次實驗: **94.45 %**)
    - 半載: **97.9 %** (此次實驗: **93.59 %**)
1. 車測中心使用的氣體燃料，為氦氫氣燃料量(30/70%):
    - 半載: **22.6 LPM**, 約為**15.7 LPM-H2** (此次實驗: **9.5 [LPM-H2]**)
    - 半載: **30 LPM**, 約為**20.9 LPM-H2** (此次實驗: **12.5 [LPM-H2]**)
2. 車測中心的燃料蒸氣的進料溫度(EVA_Out) 約落在**240-260oC** (此次實驗約落在**200-200oC**)
3. 兩次量測均未考慮環境的熱逸散